In [1]:
#import *
import numpy as np
import os
import nglview as ngl
from ase.io import read, write
WDIR='~/GroPolBul/tutorial/'
#%mkdir {WDIR}
%cd {WDIR}

#Desired polymer electrolyte system
#%mkdir PEO_CH3
%cd PEO_CH3

[Errno 2] No such file or directory: '/home/harishkemi/GroPolBul/tutorial/'
/mnt/c/Users/hargu978/Documents
[Errno 2] No such file or directory: 'PEO_CH3'
/mnt/c/Users/hargu978/Documents


/mnt/c/Users/hargu978/Documents/miniconda3/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


# Step-1: Parameterizing monomer

In [3]:
#Use shor polymer with repeat group, head and tail group
#repeat group: COCCOCCOCCOCCOC
#head and tail group: C
#Use antechaber (ambertools) to optimize and prameterise with AM1-BCC charges 
#antechamber options help:
#Usage: antechamber -i     input file name
#                   -fi    input file format
#                   -o     output file name
#                   -fo    output file format
#                   -c     charge method
#                   -nc    net molecular charge (int)
#                   -rn    residue name

!antechamber -i PEO_initial.pdb -fi pdb -o PEO.ac -fo ac -at gaff -an y -c bcc -nc 0 -rn PEO 



Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

-- Check Format for pdb File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

/bin/bash: /mnt/c/Users/hargu978/Documents/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /mnt/c/Users/hargu978/Documents/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Info: Total number of electrons: 138; net charge: 0

Running: /mnt/c/Users/hargu978/Documents/miniconda3/bin/sqm -O 

In [2]:
#Visualize the molecule
mon=read('sqm.pdb')
vi=ngl.show_ase(mon);vi.add_label(radius=2,color='black',label_type='atomindex')
vi


NGLWidget()

# Step-2: Defining the HEAD, CHAIN, and TAIL of monomer

In [5]:
#Defining CHAIN HEAD and TAIL in the monomer directory
ac=open('PEO.ac',mode='r') #Reading .ac file
[next(ac) for _ in range(2)] #Skipping first two lines of text
l=ac.readlines() #Reading lines

#Breaking the parts of monomer to CHAIN, HEAD and TAIL
#Atom index where head and tail of monomer; Check from above ngl view of mol
#Change these values accordingly
head_id=1
tail_id=7

head_omit=[3, 23, 24, 25] #Atoms to omit near head
tail_omit=[8, 32, 33, 34] #Atoms to omit near tail


In [6]:
#Write the atom indexes of CHAIN, HEAD and TAIL
chain=open('PEO.chain','w+');head=open('PEO.head','w+');tail=open('PEO.tail','w+')

chain.write('HEAD_NAME '+str(l[head_id].split()[2])+'\n')
tail.write('HEAD_NAME '+str(l[head_id].split()[2])+'\n')
chain.write('TAIL_NAME '+str(l[tail_id].split()[2])+'\n')
head.write('TAIL_NAME '+str(l[tail_id].split()[2])+'\n')

for i in range(len(head_omit)):
	chain.write('OMIT_NAME '+str(l[int(head_omit[i])].split()[2])+'\n')
	tail.write('OMIT_NAME '+str(l[int(head_omit[i])].split()[2])+'\n')
chain.write('PRE_HEAD_TYPE '+str(l[tail_id].split()[9])+'\n')
tail.write('PRE_HEAD_TYPE '+str(l[tail_id].split()[9])+'\n')
tail.write('CHARGE 0')

for i in range(len(tail_omit)):
	chain.write('OMIT_NAME '+str(l[int(tail_omit[i])].split()[2])+'\n')
	head.write('OMIT_NAME '+str(l[int(tail_omit[i])].split()[2])+'\n')
chain.write('POST_TAIL_TYPE '+str(l[head_id].split()[9])+'\n')
chain.write('CHARGE 0')
head.write('POST_TAIL_TYPE '+str(l[head_id].split()[9])+'\n')
head.write('CHARGE 0')

chain.close();head.close();tail.close()

#Use prepgen to prepare the CHAIN, HEAD and TAIL res files
#Adds dummy atoms at the desired positions
!prepgen -i PEO.ac -o PEO.prepi -f prepi -m PEO.chain -rn PEO -rf PEO.res 
!prepgen -i PEO.ac -o HPT.prepi -f prepi -m PEO.head -rn HPT -rf HPT.res 
!prepgen -i PEO.ac -o TPT.prepi -f prepi -m PEO.tail -rn TPT -rf TPT.res 


PRE_HEAD_TYPE is    c3
POST_TAIL_TYPE is    c3
Net charge of truncated molecule is     0.00
HEAD_ATOM      2   C1
TAIL_ATOM      8   C6
MAIN_CHAIN     1    2   C1
MAIN_CHAIN     2    8   C6
OMIT_ATOM      1    4   C3
OMIT_ATOM      2   24   H6
OMIT_ATOM      3   25   H7
OMIT_ATOM      4   26   H8
OMIT_ATOM      5    9   C7
OMIT_ATOM      6   33  H15
OMIT_ATOM      7   34  H16
OMIT_ATOM      8   35  H17
Number of mainchain atoms (including head and tail atom):     2
Number of omited atoms:     8
POST_TAIL_TYPE is    c3
Net charge of truncated molecule is     0.00
TAIL_ATOM      8   C6
MAIN_CHAIN     1    8   C6
OMIT_ATOM      1    9   C7
OMIT_ATOM      2   33  H15
OMIT_ATOM      3   34  H16
OMIT_ATOM      4   35  H17
Number of mainchain atoms (including head and tail atom):     1
Number of omited atoms:     4
PRE_HEAD_TYPE is    c3
Net charge of truncated molecule is     0.00
HEAD_ATOM      2   C1
MAIN_CHAIN     1    2   C1
OMIT_ATOM      1    4   C3
OMIT_ATOM      2   24   H6
OMIT_ATO

# Step-3: Build and parameterize single polymer chain

In [7]:
#Build the desired homopolymer using above prepi files
#%mkdir polymer
n_mono_repeat=5
n_mono_pol=25

repeat=" ".join(['PEO'] * int(int(n_mono_pol/n_mono_repeat)-2))
print('HPT '+str(repeat)+' TPT')

#Write the tleap input file to combine the preparatory files and build polymer chain
tleap=open('polymer/PEO_tleap.in','w+')
tleap.write('''source leaprc.gaff
loadamberprep PEO.prepi
loadamberprep HPT.prepi
loadamberprep TPT.prepi
mol = sequence {HPT '''+str(repeat)+''' TPT}
savepdb mol polymer/PEO_'''+str(n_mono_pol)+'''mer.pdb
saveamberparm mol polymer/PEO_'''+str(n_mono_pol)+'''mer.prmtop polymer/PEO_'''+str(n_mono_pol)+'''mer.inpcrd
quit''')
tleap.close()

!tleap -s -f polymer/PEO_tleap.in > polymer/PEO_tleap.out

HPT PEO PEO PEO TPT


In [10]:
#Converting AMBER to GROMACS using "intermol".
!python ../../convert.py --amb_in polymer/PEO_{n_mono_pol}mer.inpcrd polymer/PEO_{n_mono_pol}mer.prmtop --gromacs 

Traceback (most recent call last):
  File "/mnt/c/Users/hargu978/Documents/GroPolBul/tutorial/PEO_CH3/../../convert.py", line 13, in <module>
    import intermol.gromacs as gmx
  File "/mnt/c/Users/hargu978/Documents/miniconda3/lib/python3.11/site-packages/intermol/gromacs/__init__.py", line 8, in <module>
    from intermol.gromacs.gromacs_parser import load, save
  File "/mnt/c/Users/hargu978/Documents/miniconda3/lib/python3.11/site-packages/intermol/gromacs/gromacs_parser.py", line 15, in <module>
    from intermol.moleculetype import MoleculeType
  File "/mnt/c/Users/hargu978/Documents/miniconda3/lib/python3.11/site-packages/intermol/moleculetype.py", line 1, in <module>
    from intermol.orderedset import OrderedSet
  File "/mnt/c/Users/hargu978/Documents/miniconda3/lib/python3.11/site-packages/intermol/orderedset.py", line 5, in <module>
    class OrderedSet(collections.Set):
                     ^^^^^^^^^^^^^^^
AttributeError: module 'collections' has no attribute 'Set'


In [3]:
#Visualize the molecule
mol=read('polymer/PEO_25mer.pdb')
vi=ngl.show_ase(mol);#vi.add_label(radius=2,color='black',label_type='atomindex')
vi


NGLWidget()